In [2]:
#//----------------------------
#//LIBRARIES
    #Math
import math
    #Numeric Python
import numpy as np
    #Pandas (dataframes)
import pandas as pd
    #datetime for fate manipulation
from datetime import date, datetime, timedelta  
    #Regex for advanced string matching
import re
    #for time related stuff
import time
    #json library
import json
    #Analyst tools
import sys
sys.path.append('../')
from analysts_tools.growth import *
    #Procurement tools
from procurement_lib import send_slack_notification
from procurement_lib import redash
from analysts_tools.redash_methods import *

import random
import time
import datetime

import seaborn as sns
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split
from scipy.signal import savgol_filter
import statsmodels.formula.api as smf

In [3]:
clear_buyers = get_fresh_query_result("https://internal-redash.federate.frubana.com/",138674,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)

200


In [4]:
frida_products = get_fresh_query_result("https://internal-redash.federate.frubana.com/",118629,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)

200


In [5]:
city = 'SPO'
card_ids = [293209,457365,123459,197664,339060,194588,194589]
#year-month-day
todays_date = datetime.datetime.today().strftime('%Y-%m-%d')
#non_buyers_date = '2022-08-26' #1 months
customer_date_desde_query = (datetime.datetime.today() - datetime.timedelta(days=30)).strftime('%Y-%m-%d') #1 months

customer_date_desde = (datetime.datetime.today() - datetime.timedelta(days=200))#.strftime('%Y-%m-%d') #1 months

In [6]:
query ="""
with

sales as(
    select
        s.dim_submitted_date,
        su.card_id,
        case when su.source_parent_id = 0 then su.source_id else su.source_parent_id end as parent_id,
        sum((s.product_price * s.product_quantity_x_step_unit)+s.product_tax_iva) / nullif(sum(s.product_quantity_x_step_unit),0) as gross_price,
        sum((s.product_price_discount * s.product_quantity_x_step_unit)+s.product_tax_iva) / nullif(sum(s.product_quantity_x_step_unit),0) as net_price
    from dpr_sales.fact_sales s
        left join dpr_shared.dim_site site
            on s.dim_site = site.site_id
        left join dpr_shared.dim_date submit_date
            on s.dim_submitted_date = submit_date.date_id
        left join dpr_shared.dim_stock_unit su
            on s.dim_product = su.product_id
    where submit_date.full_date BETWEEN '2022-01-01' AND DATE(CURRENT_DATE)
        and s.dim_status = 1
        and site.identifier_value = '{city}'
    group by 1,2,3
)

select
    site.identifier_value as region_code,
    benchmark_day.full_date as benchmark_day,
    s.card_id,
    bc.price as benchmark_price
from dpr_pricing.snap_daily_benchmark_competitor bc
    left join dpr_shared.dim_site site
        on bc.dim_site = site.site_id
    left join dpr_shared.dim_date benchmark_day
        on bc.dim_benchmark_day = benchmark_day.date_id
    left join dpr_shared.dim_stock_unit su
        on bc.dim_stock_unit = su.stock_unit_id
    left join dpr_pricing.dim_competitor c
        on bc.dim_competitor = c.competitor_id
    left join sales s
        on s.dim_submitted_date = bc.dim_benchmark_day
        and s.parent_id = su.source_id
        and s.card_id = su.card_id
where benchmark_day.full_date BETWEEN '2022-01-01' AND DATE(CURRENT_DATE)
    and su.source_parent_id = 0
    and su.product_id != 0
    and site.identifier_value = '{city}'
order by benchmark_day desc
""".format(city=city)

df_bench = read_connection_data_warehouse.run_read_dwd_query(query)
df_bench['benchmark_day'] = pd.to_datetime(df_bench['benchmark_day']) 

df_bench.dropna(inplace=True)

In [7]:
# Group the benchmark data by product
grouped_bench = df_bench.groupby('card_id')

# Define a function to remove outliers based on the IQR method
def remove_outliers(df):
    Q1 = df['benchmark_price'].quantile(0.25)
    Q3 = df['benchmark_price'].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5*IQR
    upper_bound = Q3 + 1.5*IQR
    return df[(df['benchmark_price'] > lower_bound) & (df['benchmark_price'] < upper_bound)]

# Apply the remove_outliers function to each group and concatenate the results
df_bench_clean = pd.concat([remove_outliers(group) for name, group in grouped_bench])
df_bench = df_bench_clean.reset_index(drop=True).copy()

In [8]:
query = """
-- 93058
-- Se toma como base el query 2718
WITH 
ful AS (

SELECT DISTINCT
    bo.order_id,
    bo.submit_date
    
FROM postgres_broadleaf_federate."broadleaf.blc_order" bo
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_group" bfg        ON bfg.order_id = bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_order" bfo        ON bfo.fulfillment_group_id = bfg.fulfillment_group_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_fulfillment_group" ffg         ON ffg.fulfillment_group_id = bfg.fulfillment_group_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order" fo                      ON fo.order_id = bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_payment" bop            ON bop.order_id = bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site" s                       ON s.site_id = bo.site_disc
LEFT JOIN  postgres_broadleaf_federate."broadleaf.fb_order_type" fot                ON fot.fb_order_type_id=fo.fb_order_type_id

WHERE fo.fb_order_status_id IN (1,6,7,8)
    AND extract(year from bo.submit_date::date) > 2021
    --AND extract(month from bo.submit_date::date) = 6--2021
    AND bo.order_status = 'SUBMITTED'
    AND bfo.status NOT IN ('ARCHIVED','CANCELLED')
    AND bop.archived = 'N'
    AND (fot.name IS NULL OR fot.name <> 'REFUND')
    AND s.site_identifier_value = '{city}' --NOT IN ('MTY','PBC','GDL')
    AND bo.customer_id IN {clientes}
)--,

--more AS (
SELECT DISTINCT
    s.site_identifier_value AS city,
    bo.customer_id,
    DATE(bo.submit_date) AS submit_date,
    bs.addl_product_id AS card_id,
    bs2.name,
    AVG(boi.sale_price) AS price,
    COALESCE(AVG(boida.adjustment_value),0) AS precio_dct,
    COALESCE((AVG(boi.sale_price) - AVG(boida.adjustment_value)),AVG(boi.sale_price)) AS final_price,
    SUM(boipd.quantity*foi.step_unit) AS demand

FROM postgres_broadleaf_federate."broadleaf.blc_order" bo
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site"                     s       ON s.site_id = bo.site_disc
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_item"               boi     ON boi.order_id=bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order_item"                foi     ON boi.order_item_id= foi.order_item_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order"                     fo      ON fo.order_id = bo.order_id
INNER JOIN ful                                                                          ON ful.order_id = bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_discrete_order_item"      bdoi    ON bdoi.order_item_id = boi.order_item_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku"                      bs      ON bs.sku_id = bdoi.sku_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_product"                  bp      ON bs.addl_product_id = bp.product_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku"                      bs2     ON bs2.sku_id = bp.default_sku_id --Conectar la tarjeta con el sku que guarda la info
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category"                 bcat    ON bcat.category_id = bp.default_category_id 
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category_xref"            bcx     ON bcx.sub_category_id = bp.default_category_id AND bcx.archived='N' AND bcx.sndbx_tier is NULL
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category"                 bcat2   ON bcx.category_id = bcat2.category_id
LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_order_item_price_dtl"      boipd   ON boipd.order_item_id=boi.order_item_id
LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_order_item_dtl_adj"        boida   ON boida.order_item_price_dtl_id=boipd.order_item_price_dtl_id

WHERE fb_order_status_id IN (1,6,7,8)
    AND bo.order_status = 'SUBMITTED'
   -- FIX SUPER DESCUENTOS
    AND bcat2.category_id not in ('110873','-1000','100768','100765','100815') --ids de super descuentos en cada país
    AND bcat.name <> 'Oferton Frubana' AND bcat2.name <> 'Oferton Frubana'
    --AND bcat2.name NOT IN ('Frutas & Verduras')
GROUP BY 1,2,3,4,5
""".format(clientes=tuple(set(clear_buyers[clear_buyers.registered_city == city].customer_id.unique())), city = city)

data_ventas_aux = read_connection_data_warehouse.runQuery(query)
data_ventas_aux[["price", "precio_dct","final_price","demand"]] = data_ventas_aux[["price", "precio_dct","final_price","demand"]].astype('float32')
data_ventas_aux['submit_date'] = pd.to_datetime(data_ventas_aux['submit_date']) 

data_ventas_aux = data_ventas_aux.loc[(data_ventas_aux.demand > 0) & (data_ventas_aux.price > 0)]

In [9]:
data_ventas_aux

,city,customer_id,submit_date,card_id,name,price,precio_dct,final_price,demand
0,SPO,76409410,2023-02-07,-202912,Cenoura,3.18,0.32,2.86,12.0
1,SPO,76365771,2022-08-01,194515,Água Sanitária Triex 5L,9.39,0.00,9.39,3.0
2,SPO,89844316,2022-05-23,-202962,Chuchu,1.72,0.00,1.72,15.0
3,SPO,211332893,2023-03-14,136310,Ervilha Predilecta Lata 170g,3.79,0.00,3.79,8.0
4,SPO,51772721,2023-01-25,114310,"Ketchup Cepêra 1,01Kg",9.06,0.00,9.06,1.0
...,...,...,...,...,...,...,...,...,...
10558196,SPO,32404506,2022-01-21,-202950,Tomate Rasteiro,8.29,0.00,8.29,2.0
10558197,SPO,132113417,2022-07-22,-202950,Tomate Rasteiro,3.80,0.11,3.69,1.0
10558198,SPO,76415639,2023-02-21,-202912,Cenoura,4.50,0.00,4.50,2.0
10558199,SPO,40047021,2022-09-01,-202917,Laranja Pera,2.19,0.18,2.01,20.0


In [10]:
df = pd.merge(data_ventas_aux.loc[data_ventas_aux.card_id.isin(card_ids)], df_bench,  how='left', left_on=['card_id','submit_date'], right_on = ['card_id','benchmark_day'])
df.dropna(inplace=True)

In [11]:
df['net_PI'] = df.final_price / df.benchmark_price

In [12]:
# Group the benchmark data by product
grouped_data = df.groupby('card_id')

# Define a function to remove outliers based on the IQR method
def remove_outliers(df):
    Q1 = df['net_PI'].quantile(0.25)
    Q3 = df['net_PI'].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5*IQR
    upper_bound = Q3 + 1.5*IQR
    return df[(df['net_PI'] > lower_bound) & (df['net_PI'] < upper_bound)]

# Apply the remove_outliers function to each group and concatenate the results
df_data_clean = pd.concat([remove_outliers(group) for name, group in grouped_data])
df = df_data_clean.reset_index(drop=True).copy()

In [13]:
df = df.groupby(['customer_id', 'card_id']).filter(lambda x: len(x) > 1)

In [14]:
# Define a function to calculate the weighted average of a column
def weighted_avg(x, df):
    return (x * df['demand']).sum() / df['demand'].sum()

# Set the index to submit_date
#df2.set_index('submit_date', inplace=True)
df['order_month'] = df['submit_date'].dt.to_period('M')


# Group the data by city, customer_id, name, and month
df_monthly = df.groupby(['city', 'customer_id', 'card_id', 'name', 'order_month']).agg({
    #'price': lambda x: weighted_avg(x, df.loc[x.index]),
    #'precio_dct': lambda x: weighted_avg(x, df.loc[x.index]),
    #'final_price': lambda x: weighted_avg(x, df2.loc[x.index]),
    #'benchmark_price': lambda x: weighted_avg(x, df.loc[x.index]),
    'net_PI': lambda x: weighted_avg(x, df.loc[x.index]),
    'demand': 'sum'
}).reset_index()

In [15]:
df_monthly = df_monthly.groupby(['customer_id', 'card_id']).filter(lambda x: len(x) > 1)

In [16]:
# Group the data by customer and product
grouped = df_monthly.groupby(['customer_id', 'card_id','name'])

# Define the regression formula
formula = 'demand ~ net_PI'

# Initialize an empty DataFrame to store the results
elasticities = pd.DataFrame(columns=['customer_id', 'card_id', 'name', 'elasticity'])

# Loop through each group and fit a regression model
for group_name, group_data in grouped:
    # Check if there are multiple points
    if len(group_data) > 1:
        # Fit the regression model
        model = smf.ols(formula=formula, data=group_data).fit()
        # Calculate the elasticity of demand
        elasticity = -model.params['net_PI'] * group_data['net_PI'].mean() / group_data['demand'].mean()
    else:
        # Use the simple formula to calculate elasticity
        elasticity = group_data['net_PI'].iloc[0] / group_data['demand'].iloc[0]
    # Append the result to the DataFrame
    elasticities = elasticities.append({'customer_id': group_name[0], 'card_id': group_name[1], 'name': group_name[2], 'elasticity': elasticity}, ignore_index=True)


In [17]:
elasticities.head()

,customer_id,card_id,name,elasticity
0,19278883,197664,Coca-Cola 350ml,-2.896104
1,19355299,123459,Arroz Branco Camil Food Service 5kg,-23.451633
2,19355299,197664,Coca-Cola 350ml,-3.973352
3,19355299,293209,Óleo Vitaliv 900ml,1.608532
4,19355299,457365,Açúcar Refinado Caravelas 1kg,-9.005353


In [18]:
df_aux = df.copy()
df = df_monthly.copy()

In [19]:
df.head()

,city,customer_id,card_id,name,order_month,net_PI,demand
0,SPO,19278883,197664,Coca-Cola 350ml,2022-07,1.090909,12.0
1,SPO,19278883,197664,Coca-Cola 350ml,2022-08,1.017921,12.0
2,SPO,19278883,197664,Coca-Cola 350ml,2022-09,1.075269,12.0
3,SPO,19278883,197664,Coca-Cola 350ml,2022-10,0.960574,12.0
4,SPO,19278883,197664,Coca-Cola 350ml,2022-11,1.089479,24.0


# Aca debo empezar el experimento

In [20]:
clear_buyers[(clear_buyers.recency_days <= 20) & (clear_buyers.registered_city == city)].customer_id.nunique()

20304

In [21]:
# elasticities[(elasticities.name == sku) & 
#              (elasticities.elasticity > -1) & 
#              (elasticities.elasticity < 1) &
#              (elasticities.customer_id.isin(clear_buyers[(clear_buyers.recency_days <= 20) & (clear_buyers.registered_city == city)].customer_id.unique())) 
#              ].customer_id.nunique()

In [22]:
# elasticities[(elasticities.name == sku) & 
#              (elasticities.elasticity > 1) & 
#              #(elasticities.elasticity < 1) &
#              (elasticities.customer_id.isin(clear_buyers[(clear_buyers.recency_days <= 20) & (clear_buyers.registered_city == city)].customer_id.unique())) 
#              ].customer_id.nunique()

In [23]:
offers = data_ventas_aux.loc[(data_ventas_aux.submit_date > datetime.datetime.today() + datetime.timedelta(days=(-21))) &
                            (data_ventas_aux.card_id.isin(card_ids))].copy()

In [24]:
offers['gmv_usd'] = offers.price * offers.demand

In [25]:
offers.head()

,city,customer_id,submit_date,card_id,name,price,precio_dct,final_price,demand,gmv_usd
164,SPO,119672932,2023-08-08,457365,Açúcar Refinado Caravelas 1kg,3.46,0.07,3.39,10.0,34.599998
685,SPO,276140228,2023-08-04,197664,Coca-Cola 350ml,3.09,0.53,2.56,36.0,111.239998
711,SPO,36388454,2023-07-26,293209,Óleo Vitaliv 900ml,4.99,0.20,4.79,40.0,199.599991
712,SPO,190065156,2023-07-30,293209,Óleo Vitaliv 900ml,5.49,0.66,4.83,5.0,27.449999
729,SPO,161349318,2023-08-10,293209,Óleo Vitaliv 900ml,5.16,0.32,4.84,20.0,103.199997


In [26]:
# skus = [sku+'_INELASTIC',sku+'_ELASTIC']

In [27]:
from sklearn.model_selection import train_test_split 

def create_groups(offer):
    # if offer == sku+'_INELASTIC':
    #     customers = elasticities[(elasticities.name == sku) & 
    #          (elasticities.elasticity > -1) & 
    #          (elasticities.elasticity < 1) &
    #          (elasticities.customer_id.isin(clear_buyers[(clear_buyers.recency_days <= 20) & (clear_buyers.registered_city == city)].customer_id.unique())) 
    #          ].customer_id.unique()
    # else:
    
    customers = elasticities[(elasticities.name == offer) & 
            (elasticities.elasticity > 1) & 
            #(elasticities.elasticity < 1) &
            (elasticities.customer_id.isin(clear_buyers[(clear_buyers.recency_days <= 20) & (clear_buyers.registered_city == city)].customer_id.unique())) 
            ].customer_id.unique()
    
    if len(customers) == 0:
        return "Nada","Nada"
    else:
        print(len(customers))
    
    offer_df_aux = offers.loc[offers.name == offer].copy()
    
    customer_control, customer_test = train_test_split(customers, test_size = 0.50)
    alpha=0.01
    aux=0
    while abs(1-(offer_df_aux[offer_df_aux.customer_id.isin(customer_control)]['gmv_usd'].mean()/offer_df_aux[offer_df_aux.customer_id.isin(customer_test)]['gmv_usd'].mean())) > alpha:
        customer_control, customer_test = train_test_split(customers, test_size = 0.50)
        
        aux+=1
        if aux == 1000:
            alpha+=0.01
            aux=0
        
    return customer_control.tolist(), customer_test.tolist()

In [28]:
offers.loc[offers.card_id == card_ids[0],'name'].values[0]

'Óleo Vitaliv 900ml'

In [29]:
global_segment_list = [] #//for procesing
offer_name = []
cards = []
for i,card in enumerate(card_ids):
    offer = offers.loc[offers.card_id == card,'name'].values[0]
    print(offer)
    control,test = create_groups(offer)
    if control == "Nada" and test == "Nada":
        continue
    offer_name.append("SH_ELASTICITY_"+todays_date+"_"+city+"_"+offer)
    d = {'name':"SH_ELASTICITY_"+todays_date+"_"+city+"_"+offer, 'customersIds': list(test)}
    e = {'name':"SH_ELASTICITY_"+todays_date+"_"+city+"_"+offer+"_CONTROL", 'customersIds': list(control)}
    
    cards.append(card)
    global_segment_list.append(d)
    global_segment_list.append(e)

Óleo Vitaliv 900ml
7264
Açúcar Refinado Caravelas 1kg
2526
Arroz Branco Camil Food Service 5kg
2306
Coca-Cola 350ml
1703
Coca-Cola 2L
1055
Detergente Triex Neutro 500ml
Detergente Triex Neutro 5L
254


In [30]:
offer_name

['SH_ELASTICITY_2023-08-15_SPO_Óleo Vitaliv 900ml',
 'SH_ELASTICITY_2023-08-15_SPO_Açúcar Refinado Caravelas 1kg',
 'SH_ELASTICITY_2023-08-15_SPO_Arroz Branco Camil Food Service 5kg',
 'SH_ELASTICITY_2023-08-15_SPO_Coca-Cola 350ml',
 'SH_ELASTICITY_2023-08-15_SPO_Coca-Cola 2L',
 'SH_ELASTICITY_2023-08-15_SPO_Detergente Triex Neutro 5L']

## ADD DISCOUNT

In [35]:
dct = [5.5,3.5,7,3.5,2,5]
dct

[5.5, 3.5, 7, 3.5, 2, 5]

In [36]:
csv_file = {} 
csv_file['offer_name'] = offer_name
csv_file['offer_description'] = None
csv_file['discount'] = dct
csv_file['automatically_consider_offer'] = "true"
csv_file['start_date'] = (datetime.datetime.today() + datetime.timedelta(1)).strftime('%d-%m-%Y')
csv_file['end_date'] = (datetime.datetime.today() + datetime.timedelta(days=(8))).strftime('%d-%m-%Y')
csv_file['max_uses_per_order'] = [0]*len(offer_name)
csv_file['max_uses_per_customer'] = [0]*len(offer_name)
csv_file['customer_segment_id'] = [1]*len(offer_name) #[None]
csv_file['sku_id'] = [frida_products.loc[frida_products.id_tarjeta == sku,"skus"].values[0] for sku in cards]
global_offer_csv = pd.DataFrame(csv_file) #, index=[0])

In [37]:
global_offer_csv

,offer_name,offer_description,discount,automatically_consider_offer,start_date,end_date,max_uses_per_order,max_uses_per_customer,customer_segment_id,sku_id
0,SH_ELASTICITY_2023-08-15_SPO_Óleo Vitaliv 900ml,None,5.5,true,16-08-2023,23-08-2023,0,0,1,384239
1,SH_ELASTICITY_2023-08-15_SPO_Açúcar Refinado C...,None,3.5,true,16-08-2023,23-08-2023,0,0,1,632851
2,SH_ELASTICITY_2023-08-15_SPO_Arroz Branco Cami...,None,7.0,true,16-08-2023,23-08-2023,0,0,1,153089
3,SH_ELASTICITY_2023-08-15_SPO_Coca-Cola 350ml,None,3.5,true,16-08-2023,23-08-2023,0,0,1,244765#244764#246588
4,SH_ELASTICITY_2023-08-15_SPO_Coca-Cola 2L,None,2.0,true,16-08-2023,23-08-2023,0,0,1,452044#452045
5,SH_ELASTICITY_2023-08-15_SPO_Detergente Triex ...,None,5.0,true,16-08-2023,23-08-2023,0,0,1,261188


In [38]:
import requests
import json

url = f"https://{city}.frubana.com/api/v1/segments/create?api-key=00_growth-team-zzz-qqq_001"

payload = json.dumps(global_segment_list)


headers = {
  'Cookie': 'AWSALB=WGKrKvA1CFOY0m92xSENGAseuE4LrcfQh9y8jK/loGBJdizNm+FvjOqUqVghxdfLkH6Jsa9sfWft2NELchGAP/LNZMXMYhrOE70qDnUuOHATpdJj1UZBMEUBSIN1; AWSALBCORS=WGKrKvA1CFOY0m92xSENGAseuE4LrcfQh9y8jK/loGBJdizNm+FvjOqUqVghxdfLkH6Jsa9sfWft2NELchGAP/LNZMXMYhrOE70qDnUuOHATpdJj1UZBMEUBSIN1; AWSALB=jLY02FPysvF1vTrKLhXbk3ibzmyiAqFJ1ezYVu5PjodkU2QmPHDpRJhb9CvOlOyQ1Ix0mJR1ig+eH9zwJ9i5EOtgsgx79NSROogQb2Ua6yLX/I3bJJ6xtcVO3puI; AWSALBCORS=jLY02FPysvF1vTrKLhXbk3ibzmyiAqFJ1ezYVu5PjodkU2QmPHDpRJhb9CvOlOyQ1Ix0mJR1ig+eH9zwJ9i5EOtgsgx79NSROogQb2Ua6yLX/I3bJJ6xtcVO3puI',
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

In [39]:
sgt = [(int(x[:6]),x[7:]) for x in list(map(str.strip, response.text[32:].strip('][').replace('"', '').split(',')))]

In [40]:
sgt

[(259789, 'SH_ELASTICITY_2023-08-15_SPO_Óleo Vitaliv 900ml'),
 (259792, 'SH_ELASTICITY_2023-08-15_SPO_Óleo Vitaliv 900ml_CONTROL'),
 (259787, 'SH_ELASTICITY_2023-08-15_SPO_Açúcar Refinado Caravelas 1kg'),
 (259791,
  'SH_ELASTICITY_2023-08-15_SPO_Açúcar Refinado Caravelas 1kg_CONTROL'),
 (259788, 'SH_ELASTICITY_2023-08-15_SPO_Arroz Branco Camil Food Service 5kg'),
 (259790,
  'SH_ELASTICITY_2023-08-15_SPO_Arroz Branco Camil Food Service 5kg_CONTROL'),
 (259793, 'SH_ELASTICITY_2023-08-15_SPO_Coca-Cola 350ml'),
 (259794, 'SH_ELASTICITY_2023-08-15_SPO_Coca-Cola 350ml_CONTROL'),
 (259795, 'SH_ELASTICITY_2023-08-15_SPO_Coca-Cola 2L'),
 (259796, 'SH_ELASTICITY_2023-08-15_SPO_Coca-Cola 2L_CONTROL'),
 (259797, 'SH_ELASTICITY_2023-08-15_SPO_Detergente Triex Neutro 5L'),
 (259798, 'SH_ELASTICITY_2023-08-15_SPO_Detergente Triex Neutro 5L_CONTROL')]

In [41]:
for i in global_offer_csv.offer_name:
    for j,z in sgt:
        if z==i:
            global_offer_csv.loc[global_offer_csv.offer_name == i, ('customer_segment_id')] = j
            break

In [42]:
global_offer_csv

,offer_name,offer_description,discount,automatically_consider_offer,start_date,end_date,max_uses_per_order,max_uses_per_customer,customer_segment_id,sku_id
0,SH_ELASTICITY_2023-08-15_SPO_Óleo Vitaliv 900ml,None,5.5,true,16-08-2023,23-08-2023,0,0,259789,384239
1,SH_ELASTICITY_2023-08-15_SPO_Açúcar Refinado C...,None,3.5,true,16-08-2023,23-08-2023,0,0,259787,632851
2,SH_ELASTICITY_2023-08-15_SPO_Arroz Branco Cami...,None,7.0,true,16-08-2023,23-08-2023,0,0,259788,153089
3,SH_ELASTICITY_2023-08-15_SPO_Coca-Cola 350ml,None,3.5,true,16-08-2023,23-08-2023,0,0,259793,244765#244764#246588
4,SH_ELASTICITY_2023-08-15_SPO_Coca-Cola 2L,None,2.0,true,16-08-2023,23-08-2023,0,0,259795,452044#452045
5,SH_ELASTICITY_2023-08-15_SPO_Detergente Triex ...,None,5.0,true,16-08-2023,23-08-2023,0,0,259797,261188


In [43]:
#//--------------------------------------------
#//Exporting single file data
global_offer_csv.to_csv(f'ELASTICITY_{city}_{todays_date}_OFFER.csv', index = False)